In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from sklearn.preprocessing import MinMaxScaler

2024-04-07 01:33:10.679526: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 01:33:10.679726: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 01:33:10.844920: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
df = pd.read_csv("/kaggle/input/is-the-traffic-collision-fatal/Train.csv")
df.head()

,INDEX_,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,ObjectId
0,3387730,892658.0,2006,2006/03/11 05:00:00+00,852,BLOOR ST W,DUNDAS ST W,NaN,Major Arterial,Toronto and East York,...,Yes,NaN,NaN,NaN,88,High Park North,88,High Park North (88),D11,1
1,3387731,892658.0,2006,2006/03/11 05:00:00+00,852,BLOOR ST W,DUNDAS ST W,NaN,Major Arterial,Toronto and East York,...,Yes,NaN,NaN,NaN,88,High Park North,88,High Park North (88),D11,2
2,3388101,892810.0,2006,2006/03/11 05:00:00+00,915,MORNINGSIDE AVE,SHEPPARD AVE E,NaN,Major Arterial,Scarborough,...,Yes,Yes,NaN,NaN,146,Malvern East,132,Malvern (132),D42,3
3,3389067,893184.0,2006,2006/01/01 05:00:00+00,236,WOODBINE AVE,O CONNOR DR,NaN,Major Arterial,Toronto and East York,...,Yes,NaN,Yes,NaN,60,Woodbine-Lumsden,60,Woodbine-Lumsden (60),D55,4
4,3388102,892810.0,2006,2006/03/11 05:00:00+00,915,MORNINGSIDE AVE,SHEPPARD AVE E,NaN,Major Arterial,Scarborough,...,Yes,Yes,NaN,NaN,146,Malvern East,132,Malvern (132),D42,5


In [3]:
df.drop(['INDEX_', 'ACCNUM', 'YEAR', 'DATE', 'TIME', 'STREET1', 'STREET2', 'OFFSET', 'DISTRICT', 'WARDNUM', 'LATITUDE',
         'LONGITUDE', 'LOCCOORD', 'ACCLOC', 'FATAL_NO', 'INITDIR', 'VEHTYPE', 'MANOEUVER', 'PEDTYPE', 'PEDACT',
        'PEDCOND', 'CYCLISTYPE', 'CYCACT', 'CYCCOND', 'HOOD_158', 'NEIGHBOURHOOD_158', 'HOOD_140', 'NEIGHBOURHOOD_140',
        'DIVISION', 'ObjectId'], axis=1, inplace=True)

In [4]:
boolean_columns = ['PEDESTRIAN', 'CYCLIST', 'AUTOMOBILE', 'MOTORCYCLE', 'TRUCK', 'TRSN_CITY_VEH', 'EMERG_VEH', 'PASSENGER',
                   'SPEEDING', 'AG_DRIV', 'REDLIGHT', 'ALCOHOL', 'DISABILITY']

df[boolean_columns] = df[boolean_columns].fillna("No")
df

,ROAD_CLASS,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,...,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY
0,Major Arterial,Traffic Signal,Clear,Daylight,Dry,Fatal,Pedestrian Collisions,Driver,unknown,NaN,...,No,No,No,No,No,No,Yes,No,No,No
1,Major Arterial,Traffic Signal,Clear,Daylight,Dry,Fatal,Pedestrian Collisions,Pedestrian,65 to 69,Fatal,...,No,No,No,No,No,No,Yes,No,No,No
2,Major Arterial,Traffic Signal,Clear,Daylight,Dry,Fatal,Turning Movement,Motorcycle Driver,45 to 49,Fatal,...,Yes,No,No,No,No,No,Yes,Yes,No,No
3,Major Arterial,No Control,Clear,Dark,Wet,Non-Fatal Injury,Approaching,Passenger,50 to 54,Major,...,No,No,No,No,Yes,Yes,Yes,No,Yes,No
4,Major Arterial,Traffic Signal,Clear,Daylight,Dry,Fatal,Turning Movement,Driver,unknown,NaN,...,Yes,No,No,No,No,No,Yes,Yes,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,Major Arterial,Traffic Signal,Clear,"Dusk, artificial",Dry,Non-Fatal Injury,Turning Movement,Passenger,25 to 29,NaN,...,No,No,No,No,Yes,No,Yes,No,No,No
14996,Major Arterial,Traffic Signal,Clear,"Dusk, artificial",Dry,Non-Fatal Injury,Turning Movement,Passenger,0 to 4,Minor,...,No,No,No,No,Yes,No,Yes,No,No,No
14997,Major Arterial,Traffic Signal,Clear,"Dusk, artificial",Dry,Non-Fatal Injury,Turning Movement,Passenger,5 to 9,NaN,...,No,No,No,No,Yes,No,Yes,No,No,No
14998,Major Arterial,Traffic Signal,Clear,"Dusk, artificial",Dry,Non-Fatal Injury,Turning Movement,Passenger,5 to 9,NaN,...,No,No,No,No,Yes,No,Yes,No,No,No


In [5]:
from sklearn.preprocessing import LabelEncoder
for column in df:
    if df[column].dtype == np.number:
        continue
    df[column] = LabelEncoder().fit_transform(df[column])
df

/tmp/ipykernel_18/1539703795.py:3: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if df[column].dtype == np.number:
/tmp/ipykernel_18/1539703795.py:3: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if df[column].dtype == np.number:
/tmp/ipykernel_18/1539703795.py:3: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if df[column].dtype == np.number:
/tmp/ipykernel_18/1539703795.py:3: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if df[column].dtype == np.number:
/tmp/ipykernel_18/1539703795.py:3: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprec

,ROAD_CLASS,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,...,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY
0,5,8,0,4,0,0,4,2,20,4,...,0,0,0,0,0,0,1,0,0,0
1,5,8,0,4,0,0,4,11,13,0,...,0,0,0,0,0,0,1,0,0,0
2,5,8,0,4,0,0,9,6,8,0,...,1,0,0,0,0,0,1,1,0,0
3,5,0,0,0,8,1,1,10,10,1,...,0,0,0,0,1,1,1,0,1,0
4,5,8,0,4,0,0,9,2,20,4,...,1,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,5,8,0,7,0,1,9,10,4,4,...,0,0,0,0,1,0,1,0,0,0
14996,5,8,0,7,0,1,9,10,0,3,...,0,0,0,0,1,0,1,0,0,0
14997,5,8,0,7,0,1,9,10,9,4,...,0,0,0,0,1,0,1,0,0,0
14998,5,8,0,7,0,1,9,10,9,4,...,0,0,0,0,1,0,1,0,0,0


In [6]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

df[df.columns] = scaler.fit_transform(df[df.columns])

df

,ROAD_CLASS,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,...,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY
0,0.555556,0.8,0.0,0.500,0.000000,0.0,0.444444,0.111111,1.00,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.555556,0.8,0.0,0.500,0.000000,0.0,0.444444,0.611111,0.65,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.555556,0.8,0.0,0.500,0.000000,0.0,1.000000,0.333333,0.40,0.00,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,0.555556,0.0,0.0,0.000,0.888889,1.0,0.111111,0.555556,0.50,0.25,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
4,0.555556,0.8,0.0,0.500,0.000000,0.0,1.000000,0.111111,1.00,1.00,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.555556,0.8,0.0,0.875,0.000000,1.0,1.000000,0.555556,0.20,1.00,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
14996,0.555556,0.8,0.0,0.875,0.000000,1.0,1.000000,0.555556,0.00,0.75,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
14997,0.555556,0.8,0.0,0.875,0.000000,1.0,1.000000,0.555556,0.45,1.00,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
14998,0.555556,0.8,0.0,0.875,0.000000,1.0,1.000000,0.555556,0.45,1.00,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [7]:
'''corr_df = df.corr()
corr_df'''

'corr_df = df.corr()\ncorr_df'

In [8]:
'''weak_corr_df = corr_df[(corr_df["ACCLASS"] < 0.00) & (corr_df["ACCLASS"] > -0.00)]
weak_corr_df["ACCLASS"]'''

'weak_corr_df = corr_df[(corr_df["ACCLASS"] < 0.00) & (corr_df["ACCLASS"] > -0.00)]\nweak_corr_df["ACCLASS"]'

In [9]:
'''high_corr_df = df.copy()
for column in df.columns:
  if column in weak_corr_df.index:
    high_corr_df = high_corr_df.drop(column, axis=1)
  else:
    continue
high_corr_df.corr()["ACCLASS"]'''

'high_corr_df = df.copy()\nfor column in df.columns:\n  if column in weak_corr_df.index:\n    high_corr_df = high_corr_df.drop(column, axis=1)\n  else:\n    continue\nhigh_corr_df.corr()["ACCLASS"]'

In [10]:
from sklearn.model_selection import train_test_split
X = df.drop('ACCLASS', axis=1)
y = df['ACCLASS']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [11]:
shape = X_train.shape
shape[1]

24

In [12]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(shape[1],)))
model.add(Dense(units=16, activation='relu', input_shape=(shape[1],)))
model.add(Dense(units=4, activation='relu', input_shape=(shape[1],)))
model.add(Dense(1, activation='sigmoid'))
adam = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=["accuracy"])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8515 - loss: 0.4836
Epoch 2/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8693 - loss: 0.3633
Epoch 3/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8665 - loss: 0.3525
Epoch 4/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8964 - loss: 0.3062
Epoch 5/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9120 - loss: 0.2749
Epoch 6/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9161 - loss: 0.2520
Epoch 7/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9134 - loss: 0.2526
Epoch 8/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9164 - loss: 0.2424
Epoch 9/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9107 - loss: 0.2529
Epoch 10/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9151 - loss: 0.2427
Epoch 11/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9182 - loss: 0.2374
Epoch 12/100
352/352 ━━━━━━━━━━━━━━━━━━━━

In [14]:
loss_and_metrics = model.evaluate(X_test, y_test)
print(loss_and_metrics)
print('Loss = ',loss_and_metrics[0])
print('Accuracy = ',loss_and_metrics[1])

118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9140 - loss: 0.2555
[0.25562095642089844, 0.9120000004768372]
Loss =  0.25562095642089844
Accuracy =  0.9120000004768372


In [15]:
test_df = pd.read_csv("/kaggle/input/is-the-traffic-collision-fatal/Test.csv")
test_df.head()

,INDEX_,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,ObjectId
0,81451576,NaN,2019,2019/09/03 04:00:00+00,1903,KING ST W,PORTLAND ST,30 m East of,Major Arterial,Toronto and East York,...,NaN,NaN,NaN,NaN,164,Wellington Place,77,Waterfront Communities-The Island (77),D14,15001
1,81451577,NaN,2019,2019/09/03 04:00:00+00,1903,KING ST W,PORTLAND ST,30 m East of,Major Arterial,Toronto and East York,...,NaN,NaN,NaN,NaN,164,Wellington Place,77,Waterfront Communities-The Island (77),D14,15002
2,81262860,NaN,2019,2019/03/04 05:00:00+00,1755,GARDINER LAKESHORE E,LOWER JARVIS ST,55 m West of,Expressway Ramp,Toronto and East York,...,NaN,NaN,NaN,NaN,166,St Lawrence-East Bayfront-The Islands,77,Waterfront Communities-The Island (77),D51,15003
3,81452521,NaN,2019,2019/09/03 04:00:00+00,1910,LANSDOWNE AVE,WALLACE AVE,27 m South of,Minor Arterial,Etobicoke York,...,NaN,NaN,NaN,NaN,171,Junction-Wallace Emerson,93,Dovercourt-Wallace Emerson-Junction (93),D11,15004
4,81452522,NaN,2019,2019/09/03 04:00:00+00,1910,LANSDOWNE AVE,WALLACE AVE,27 m South of,Minor Arterial,Etobicoke York,...,NaN,NaN,NaN,NaN,171,Junction-Wallace Emerson,93,Dovercourt-Wallace Emerson-Junction (93),D11,15005


In [16]:
object_id = test_df['ObjectId']

test_df.drop(['INDEX_', 'ACCNUM', 'YEAR', 'DATE', 'TIME', 'STREET1', 'STREET2', 'OFFSET', 'DISTRICT', 'WARDNUM', 'LATITUDE',
         'LONGITUDE', 'LOCCOORD', 'ACCLOC', 'FATAL_NO', 'INITDIR', 'VEHTYPE', 'MANOEUVER', 'PEDTYPE', 'PEDACT',
        'PEDCOND', 'CYCLISTYPE', 'CYCACT', 'CYCCOND', 'HOOD_158', 'NEIGHBOURHOOD_158', 'HOOD_140', 'NEIGHBOURHOOD_140',
        'DIVISION', 'ObjectId'], axis=1, inplace=True)
test_df

,ROAD_CLASS,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,IMPACTYPE,INVTYPE,INVAGE,INJURY,DRIVACT,...,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY
0,Major Arterial,Traffic Signal,Clear,Dark,Dry,Cyclist Collisions,Cyclist,45 to 49,Major,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Major Arterial,Traffic Signal,Clear,Dark,Dry,Cyclist Collisions,Driver,30 to 34,NaN,Driving Properly,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Expressway Ramp,No Control,Clear,Daylight,Dry,Pedestrian Collisions,Driver,25 to 29,NaN,Driving Properly,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Minor Arterial,No Control,Clear,Daylight,Dry,Pedestrian Collisions,Driver,25 to 29,NaN,Lost control,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Minor Arterial,No Control,Clear,Daylight,Dry,Pedestrian Collisions,Pedestrian,25 to 29,Major,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3189,Local,Stop Sign,Clear,Dark,Wet,Pedestrian Collisions,Pedestrian,5 to 9,Minimal,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN
3190,Local,Stop Sign,Clear,Dark,Wet,Pedestrian Collisions,Pedestrian,0 to 4,Minor,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN
3191,Local,Stop Sign,Clear,Dark,Wet,Pedestrian Collisions,Pedestrian,0 to 4,Minimal,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN
3192,Major Arterial,No Control,Rain,"Dark, artificial",Wet,Pedestrian Collisions,Driver,35 to 39,NaN,Driving Properly,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
boolean_columns = ['PEDESTRIAN', 'CYCLIST', 'AUTOMOBILE', 'MOTORCYCLE', 'TRUCK', 'TRSN_CITY_VEH', 'EMERG_VEH', 'PASSENGER',
                   'SPEEDING', 'AG_DRIV', 'REDLIGHT', 'ALCOHOL', 'DISABILITY']

test_df[boolean_columns] = test_df[boolean_columns].fillna("No")
test_df

,ROAD_CLASS,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,IMPACTYPE,INVTYPE,INVAGE,INJURY,DRIVACT,...,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY
0,Major Arterial,Traffic Signal,Clear,Dark,Dry,Cyclist Collisions,Cyclist,45 to 49,Major,NaN,...,No,No,No,No,No,No,No,No,No,No
1,Major Arterial,Traffic Signal,Clear,Dark,Dry,Cyclist Collisions,Driver,30 to 34,NaN,Driving Properly,...,No,No,No,No,No,No,No,No,No,No
2,Expressway Ramp,No Control,Clear,Daylight,Dry,Pedestrian Collisions,Driver,25 to 29,NaN,Driving Properly,...,No,No,No,No,No,No,No,No,No,No
3,Minor Arterial,No Control,Clear,Daylight,Dry,Pedestrian Collisions,Driver,25 to 29,NaN,Lost control,...,No,No,No,No,No,No,No,No,No,No
4,Minor Arterial,No Control,Clear,Daylight,Dry,Pedestrian Collisions,Pedestrian,25 to 29,Major,NaN,...,No,No,No,No,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3189,Local,Stop Sign,Clear,Dark,Wet,Pedestrian Collisions,Pedestrian,5 to 9,Minimal,NaN,...,No,No,No,No,No,No,Yes,No,No,No
3190,Local,Stop Sign,Clear,Dark,Wet,Pedestrian Collisions,Pedestrian,0 to 4,Minor,NaN,...,No,No,No,No,No,No,Yes,No,No,No
3191,Local,Stop Sign,Clear,Dark,Wet,Pedestrian Collisions,Pedestrian,0 to 4,Minimal,NaN,...,No,No,No,No,No,No,Yes,No,No,No
3192,Major Arterial,No Control,Rain,"Dark, artificial",Wet,Pedestrian Collisions,Driver,35 to 39,NaN,Driving Properly,...,No,No,No,No,No,No,No,No,No,No


In [18]:
from sklearn.preprocessing import LabelEncoder
for column in test_df:
    if test_df[column].dtype == np.number:
        continue
    test_df[column] = LabelEncoder().fit_transform(test_df[column])
test_df

/tmp/ipykernel_18/2886290627.py:3: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if test_df[column].dtype == np.number:
/tmp/ipykernel_18/2886290627.py:3: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if test_df[column].dtype == np.number:
/tmp/ipykernel_18/2886290627.py:3: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if test_df[column].dtype == np.number:
/tmp/ipykernel_18/2886290627.py:3: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if test_df[column].dtype == np.number:
/tmp/ipykernel_18/2886290627.py:3: DeprecationWarning: Converting `np.inexact` or `np.floating` 

,ROAD_CLASS,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,IMPACTYPE,INVTYPE,INVAGE,INJURY,DRIVACT,...,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY
0,5,4,0,0,0,2,0,8,1,12,...,0,0,0,0,0,0,0,0,0,0
1,5,4,0,0,0,2,2,5,4,1,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,4,0,4,2,4,4,1,...,0,0,0,0,0,0,0,0,0,0
3,7,0,0,4,0,4,2,4,4,8,...,0,0,0,0,0,0,0,0,0,0
4,7,0,0,4,0,4,10,4,1,12,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3189,4,2,0,0,7,4,10,9,2,12,...,0,0,0,0,0,0,1,0,0,0
3190,4,2,0,0,7,4,10,0,3,12,...,0,0,0,0,0,0,1,0,0,0
3191,4,2,0,0,7,4,10,0,2,12,...,0,0,0,0,0,0,1,0,0,0
3192,5,0,5,1,7,4,2,6,4,1,...,0,0,0,0,0,0,0,0,0,0


In [19]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

test_df[test_df.columns] = scaler.fit_transform(test_df[test_df.columns])

test_df

,ROAD_CLASS,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,IMPACTYPE,INVTYPE,INVAGE,INJURY,DRIVACT,...,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY
0,0.625,0.666667,0.000,0.000,0.000,0.2,0.000000,0.40,0.25,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.625,0.666667,0.000,0.000,0.000,0.2,0.142857,0.25,1.00,0.083333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.250,0.000000,0.000,0.500,0.000,0.4,0.142857,0.20,1.00,0.083333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.875,0.000000,0.000,0.500,0.000,0.4,0.142857,0.20,1.00,0.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.875,0.000000,0.000,0.500,0.000,0.4,0.714286,0.20,0.25,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3189,0.500,0.333333,0.000,0.000,0.875,0.4,0.714286,0.45,0.50,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3190,0.500,0.333333,0.000,0.000,0.875,0.4,0.714286,0.00,0.75,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3191,0.500,0.333333,0.000,0.000,0.875,0.4,0.714286,0.00,0.50,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3192,0.625,0.000000,0.625,0.125,0.875,0.4,0.142857,0.30,1.00,0.083333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
test_pred = model.predict(test_df)
test_pred

100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([[0.999703  ],
       [0.98868257],
       [0.84330755],
       ...,
       [0.9981034 ],
       [0.8028426 ],
       [0.99844724]], dtype=float32)

In [21]:
import numpy as np
test_pred_labels = np.where(test_pred < 0.5, 'Fatal', 'Non-Fatal Injury')
test_pred_labels

array([['Non-Fatal Injury'],
       ['Non-Fatal Injury'],
       ['Non-Fatal Injury'],
       ...,
       ['Non-Fatal Injury'],
       ['Non-Fatal Injury'],
       ['Non-Fatal Injury']], dtype='<U16')

In [22]:
test_pd = test_pred_labels
print(test_pd)

[['Non-Fatal Injury']
 ['Non-Fatal Injury']
 ['Non-Fatal Injury']
 ...
 ['Non-Fatal Injury']
 ['Non-Fatal Injury']
 ['Non-Fatal Injury']]


In [23]:
dff = pd.DataFrame(test_pd)
dff = dff.rename(columns={0: "ACCLASS"})
dff

,ACCLASS
0,Non-Fatal Injury
1,Non-Fatal Injury
2,Non-Fatal Injury
3,Non-Fatal Injury
4,Non-Fatal Injury
...,...
3189,Non-Fatal Injury
3190,Non-Fatal Injury
3191,Non-Fatal Injury
3192,Non-Fatal Injury


In [24]:
fdf = pd.DataFrame({'ObjectId': object_id})
final_df = fdf.join(dff)
final_df

,ObjectId,ACCLASS
0,15001,Non-Fatal Injury
1,15002,Non-Fatal Injury
2,15003,Non-Fatal Injury
3,15004,Non-Fatal Injury
4,15005,Non-Fatal Injury
...,...,...
3189,18190,Non-Fatal Injury
3190,18191,Non-Fatal Injury
3191,18192,Non-Fatal Injury
3192,18193,Non-Fatal Injury


In [25]:
final_df.groupby('ACCLASS').count()

,ObjectId
ACCLASS,
Fatal,349
Non-Fatal Injury,2845


In [26]:
final_df.to_csv('res.csv', index=False)